In [10]:
import json
import os
import io
from Waveforms import *
from numpy.random import default_rng
from numpy.random import RandomState
rng = default_rng(seed=1234)
prng = RandomState(1234)

ROOT_PATH = current_folder = globals()['_dh'][0]
JSON_SAVE_PATH = os.path.join(ROOT_PATH, "json")
venv_activate_path = "/home/mcheung1/Ringdown/jaxqualin/bin/activate"
eff_mode_search_path = os.path.join(ROOT_PATH, "eff_mode_search.py")

slurm_dir = os.path.join(ROOT_PATH, "slurm_scripts")

In [11]:
# https://stackoverflow.com/questions/57269741/typeerror-object-of-type-ndarray-is-not-json-serializable
class NumpyEncoder(json.JSONEncoder):

    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32,
                              np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [12]:
af = 0.7
batchname = f"eff3_a_{af}_overtone_4"
batch_len = 10
setting_name = "normal3"
output = io.StringIO()
# N_list = [5, 6, 7, 8, 9, 10]
# print(N_list, file = output)
# N_list_str = output.getvalue().rstrip()

delay = False

load_run = False
load_runname = "eff2"
load_pickle = True

# kwargs_in = {} # depreciated
# output = io.StringIO()
# print(kwargs_in, file = output)
# if len(kwargs_in) == 0:
#     kwargs_in_str = ""
# else:
#     kwargs_in_str = '"' + output.getvalue().rstrip() + '"'

In [13]:
inject_params_base = {
    'l' : 2,
    'm' : 2,
    'Mf' : 1,
    'af' : af,
    'inj_dict' : {
        "2.2.0": (1, 1),
        "2.2.1": (3, 4),
        "2.2.2": (10, 5),
        "2.2.3": (20, 3),
        "2.2.4": (20, 3),
        # "2.2.5": (20, 3),
        # "2.2.6": (20, 3),
        # "2.2.7": (20, 3),
        "2.1.0": (8e-4, 3.5),
        "3.3.0": (0.1, 1),
        "3.2.0": (0.05, 1.5),
        "4.2.0": (4e-4, 0.5),
        "4.4.0": (1e-3, 2),
        "2.-2.0": (1e-5, 3),
        "3.-2.0": (5e-6, 2.5),
        "constant": (4e-6, 3),
    },
    'relevant_lm_list' : [(2, 2), (3, 2), (3, 3), (4, 4), (2, 1), (2, 0)],
    'noise_sig' : 5e-7,
    'time' : np.linspace(0, 150, num = 1501)
}

prng = RandomState(1234)
inject_params_base['noise_arr'] = prng.normal(0, 
                            inject_params_base['noise_sig'],
                            len(inject_params_base['time']))

randomize_params = {
    'noise_random' : True
}

phase_random_range = (0, 2*np.pi)
overtone1_random_range = (3, 5)
overtone2_random_range = (8, 12)
overtone3_random_range = (12, 24)
overtone4_random_range = (20, 30)
overtone5_random_range = (20, 30)
overtone6_random_range = (5, 15)
overtone7_random_range = (2, 4)
spheroidal_random_range = (1e-3, 1e-2)
recoil_random_range = (1e-4, 1e-2)
retro_random_range = (1e-5, 1e-3)
inj_dict_randomize = {
    "2.2.1": (overtone1_random_range, phase_random_range),
    "2.2.2": (overtone2_random_range, phase_random_range),
    "2.2.3": (overtone3_random_range, phase_random_range),
    "2.2.4": (overtone4_random_range, phase_random_range),
    # "2.2.5": (overtone5_random_range, phase_random_range),
    # "2.2.6": (overtone6_random_range, phase_random_range),
    # "2.2.7": (overtone7_random_range, phase_random_range),
    "2.1.0": (recoil_random_range, phase_random_range),
    "3.3.0": (recoil_random_range, phase_random_range),
    "3.2.0": (spheroidal_random_range, phase_random_range),
    "4.2.0": (spheroidal_random_range, phase_random_range),
    "4.4.0": (recoil_random_range, phase_random_range),
    "2.-2.0": (retro_random_range, phase_random_range),
    "3.-2.0": (retro_random_range, phase_random_range),
    "constant": ((1e-6, 5e-5), phase_random_range),
}

amp_order = [["3.2.0", "4.2.0"], ["2.-2.0", "3.-2.0"]]


In [ ]:
inject_params_full = {}

if load_run:
    with open(f"{JSON_SAVE_PATH}/{load_runname}.json", 'r') as f:
            inject_params_full_load = json.load(f)

    if batch_len != len(inject_params_full_load):
        raise ValueError

    for i in range(batch_len):
        inject_params_full[f"{batchname}_{i:03d}"] = inject_params_full_load[f"{load_runname}_{i:03d}"]
else:
    for i in range(batch_len):
        inject_params = make_random_inject_params(inject_params_base, randomize_params, inj_dict_randomize, amp_order)
        inject_params_full[f"{batchname}_{i:03d}"] = inject_params
    
with open(f"{JSON_SAVE_PATH}/{batchname}.json", 'w') as f:
    json.dump(inject_params_full, f, indent = 4, cls=NumpyEncoder)


In [ ]:
os.makedirs(f"{slurm_dir}/slurm_out/{batchname}/", exist_ok=True)

In [7]:
if delay:
    nd_string = ''
else:
    nd_string = '-nd'

if load_pickle:
    lp_string = ''
else:
    lp_string = '-l'

In [8]:
sbatch_script = f"""#!/bin/bash
#SBATCH --job-name="{batchname}"
#SBATCH --output="{slurm_dir}/slurm_out/{batchname}/{batchname}.%A.%a.out"
#SBATCH --partition=shared
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=2Gb
#SBATCH --account=cit122
#SBATCH -t 24:00:00
#SBATCH --mail-type=ALL
#SBATCH --mail-user=hcheung5@jhu.edu
#SBATCH --array=0-{batch_len-1}

pwd; hostname; date

source {venv_activate_path}

python {eff_mode_search_path} {batchname} $SLURM_ARRAY_TASK_ID {setting_name} {nd_string} {lp_string}

date
"""

In [9]:
with open(slurm_dir + "/" + batchname + ".sbatch", 'w') as f:
    f.write(sbatch_script)